<a href="https://colab.research.google.com/github/maschu09/mless/blob/main/time_series_forecasting/1_Download_Preprocess_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data download and preparation

In the following, we will set-up a typical data preprocessing workflow for (environmental) timeseries data. It is quite common that such data can be obtained via a web interface (REST API) and that the output from this API requires substantial reformatting and rearranging before the data can be imported into analysis or forecasting models. Typical issues encountered include:
* time series at different stations (and for different parameters) vary in length
* timeseries have some or many missing values, which are either absent from the data series or encoded as special values
* data vary in quality (some datasets may have jumps or awkward "features" that cannot be explained by natural phenomena)
* metadata to inform the data selection are missing ort difficult to access/process

In this notebook, we try to keep things relatively simple and straightforward while providing a set of tools that can also be used in more meaningful applications.

The default use case on which this notebook builds is to extract timeseries of the variable "temperature" from a handful of measurement locations, store intermediate results to avoid repeated downloads, and package the data into a single CSV file for input in the statistical analyses and ML models shown in the subsequent notebooks. The code will also run if you extract several variables at once or increase the number of stations. However, for massive data downloads, you will need to rewrite the code to be more efficient and add some monitoring of your downloads.

> Please note: In case of runtime loss or a need to run any segmented sections of the code make sure to run all the housekeeping cells before it






## Initial Setup and Data download
> This section downloads example data from TOAR for 5 stations in Germany. Refer to [TOAR Quick UserGuide](https://toar-data.fz-juelich.de/sphinx/TOAR_UG_Vol02_Quick_Start/build/html/examples.html) examples to better understand data structuring in TOAR that is used in the below snippet to download examples.

<p align="center">
  <img src="https://github.com/maschu09/mless/blob/main/time_series_forecasting/TOARFetchingBlockDiag.png?raw=1" alt="TOAR Block Diag" width="600"/>
</p>


### Housekeeping: Initial setup, declarations and method definitions




In [414]:
# Most notebook servers like google collab should have these packages pre-installed
# In such cases this is just a sanity check
#!pip install pandas numpy requests tensorflow

In [1]:
import requests
import json
import pandas as pd
import os
import csv
from datetime import datetime


#### Alternative

In [4]:
# Mount google drive when working in colab
hasCOLAB = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False
if hasCOLAB:
  from google.colab import drive
  drive.mount('/content/drive')
  BASEPATH = '/content/drive/MyDrive'
else:
  BASEPATH = '.'

In [6]:
#!ls drive/MyDrive

In [8]:
# Global constants
TIMESERIES_DATA_DIR = BASEPATH + "/timeseries_data/"
TIMESERIES_CSV_DIR = os.path.join(TIMESERIES_DATA_DIR, "toar_csv_timeseries")
TIMESERIES_ID_FILE = os.path.join(TIMESERIES_DATA_DIR, "timeseriesIDs.json")
MIN_FILE_SIZE_BYTES = 100
group_columns = ['station_code', 'latitude', 'longitude']

os.makedirs(TIMESERIES_DATA_DIR, exist_ok=True)
os.makedirs(TIMESERIES_CSV_DIR, exist_ok=True)

### Custom data selection for experiments

The station codes in the example snippet below are checked for a common daterange for the chosen variables and offer reasonably complete timeseries. If you prefer to experiment with other datasets, consult the documentation on the [Search API](https://toar-data.fz-juelich.de/api/v2/#search-combined-endpoint-of-stations-and-timeseries).

😈 **Question 1:** What are common reasons for gaps in observational data records, such as from TOAR?


In [11]:
# German stations with good data coverage
station_codes = ["DENW094", "DEBW073","DEHE020"]
variable_columns = ["temp"]
# station_codes = ["DENW094"]
# variable_columns = ["no2", "temp", "o3", "no", "press"]


😈 **Task 1:** Explore the `station_codes` variable and try changing the station(s) to a different region.

Below methods each have appropriate documentation and comments to illustrate the logical flow *(they are placed with enough safeguards against both the API and optimized to avoid re-downloads when interupted during partial downloads to accomodate any loss of runtime on platforms like google colab)* and briefly described here for ease of use

😈 **Task 2:** Inspect the function `pivot_handle()`. What does it return, and why is pivoting important for time series analysis?


In [14]:
def load_existing_timeseries_ids():
    """
    Load existing timeseries IDs from a JSON file.

    Returns:
        dict: A dictionary containing stored timeseries metadata.
    """
    return json.load(open(TIMESERIES_ID_FILE, 'r')) if os.path.exists(TIMESERIES_ID_FILE) else {}

def save_timeseries_ids(timeseries_data):
    """
    Save timeseries metadata to a JSON file.

    Args:
        timeseries_data (dict): A dictionary containing timeseries metadata.
    """
    json.dump(timeseries_data, open(TIMESERIES_ID_FILE, 'w'), indent=4)

def fetch_timeseries_data(station_codes, existing_timeseries, variable_columns):
    """
    Fetch timeseries metadata for given station codes, filtering by specified variables.

    Args:
        station_codes (list): List of station codes to fetch data for.
        existing_timeseries (dict): Dictionary of previously fetched timeseries metadata.
        variable_columns (list): List of variable names to retain.

    Returns:
        dict: Updated dictionary containing filtered timeseries metadata.
    """
    base_url = "http://toar-data.fz-juelich.de/api/v2/search/?codes="
    unique_entries = existing_timeseries.copy()
    processed_station_codes = {details['station_code'] for details in existing_timeseries.values()}

    for code in station_codes:
        if code in processed_station_codes:
            print(f"\t\tStation {code} is already processed, skipping.")
            continue

        response = requests.get(base_url + code, timeout=1000)
        if response.status_code == 200:
            for entry in response.json():
                if (variable_name := entry.get('variable', {}).get('name')) in variable_columns:
                    timeseries_id = entry.get('id')
                    if timeseries_id not in unique_entries:
                        unique_entries[timeseries_id] = {
                            'data_start_date': entry.get('data_start_date'),
                            'data_end_date': entry.get('data_end_date'),
                            'variable_name': variable_name,
                            'station_code': code,
                            'latitude': entry.get('station', {}).get('coordinates', {}).get('lat'),
                            'longitude': entry.get('station', {}).get('coordinates', {}).get('lng'),
                        }
        else:
            print(f"\t\tFailed to fetch data for station {code}. Status code: {response.status_code}")
    return unique_entries

def pivot_handle(dfs, metadata_columns, variable_columns):
    """
    Pivot and structure the timeseries dataframe for sequential data analysis.

    Args:
        dfs (pd.DataFrame): Dataframe containing timeseries data.
        metadata_columns (list): List of metadata column names.
        variable_columns (list): List of variable names to include.

    Returns:
        pd.DataFrame: Processed dataframe with pivoted structure.
    """
    pivot_df = dfs.pivot_table(index='datetime', columns='variable_name', values='value', aggfunc='mean')
    pivot_df.reset_index(inplace=True)

    print(f"Station {dfs['station_code'].unique()} min time: {dfs['datetime'].min()}, max time: {dfs['datetime'].max()}, hours between: {(dfs['datetime'].max() - dfs['datetime'].min()) / pd.Timedelta(hours=1):.2f}")
    reference_index = pd.date_range(start=dfs['datetime'].min(), end=dfs['datetime'].max(), freq="h", tz="UTC")
    reference_df = pd.DataFrame({'datetime': reference_index})

    pivot_df_ = reference_df.merge(pivot_df, on='datetime', how='left')

    for col in metadata_columns:
        if dfs[col].notna().any():
            value = dfs[col].dropna().iloc[0]
            pivot_df_.insert(0, col, value)
        else:
            print(f"Station {dfs['station_code'].unique()}: metadata {col} has no value")

    return pivot_df_

def download_csv_data(timeseries_data, variable_columns):
    """
    Download and process CSV data for each timeseries ID.

    Args:
        timeseries_data (dict): Dictionary containing timeseries metadata.
        variable_columns (list): List of variable names to process.

    Returns:
        pd.DataFrame: Combined dataframe of all timeseries data.
    """
    dataframes = []
    metadata_columns = ['station_code', 'latitude', 'longitude']

    for ts_id, details in timeseries_data.items():
        csv_path = os.path.join(TIMESERIES_CSV_DIR, f"{ts_id}.csv")

        if os.path.exists(csv_path) and os.path.getsize(csv_path) > MIN_FILE_SIZE_BYTES:
            print(f"\tCSV already exists for timeseries ID {ts_id}, skipping download.")
        else:
            print(f"\tDownloading data for timeseries ID {ts_id}")
            url = f"http://toar-data.fz-juelich.de/api/v2/data/timeseries/{ts_id}?format=csv"
            try:
                response = requests.get(url, stream=True, timeout=1000)
                response.raise_for_status()
                with open(csv_path, 'wb') as file:
                    file.writelines(response.iter_content(chunk_size=8192))
                print(f"\t\tRaw data CSV of {ts_id} saved: {csv_path}")
            except requests.exceptions.RequestException as e:
                print(f"\t\tFailed to download data for timeseries ID {ts_id}. Error: {e}")
                continue

        try:
            df = pd.read_csv(csv_path, skiprows=lambda i: i < next(i for i, line in enumerate(open(csv_path)) if line.startswith('datetime')), low_memory=False)
            df['datetime'] = pd.to_datetime(df['datetime'], format='mixed')
            df[['variable_name', 'station_code', 'latitude', 'longitude']] = details['variable_name'], details['station_code'], details['latitude'], details['longitude']
            print(f"Dataframe for timeseries ID {ts_id} loaded successfully with shape {df.shape}")
            dataframes.append(pivot_handle(df, metadata_columns, variable_columns))
        except (pd.errors.EmptyDataError, pd.errors.ParserError) as e:
            print(f"\tError processing CSV for timeseries ID {ts_id}: {e}")
            continue

    return pd.concat(dataframes, ignore_index=True).sort_values(by=['station_code', 'datetime']) if dataframes else pd.DataFrame()

### Download via REST API

😈 **Task 3:** Try downloading a different variable or add another pollutant (e.g., `so2`). What changes?


In [432]:
# Load existing timeseries IDs from json to skip calls to TOAR
existing_timeseries = load_existing_timeseries_ids()

timeseries_data = fetch_timeseries_data(station_codes, existing_timeseries,variable_columns)
print(f"\t Number of time series meta data fetched : {len(timeseries_data)}")

# save existing timeseries IDs as json to reduce calls to TOAR in future
save_timeseries_ids(timeseries_data)

dataframes = download_csv_data(timeseries_data,variable_columns)
print(f"\t Total dataFrames processed : {len(dataframes)} and shape of first dataframe {dataframes.shape}.")

dataframes.head()

		Station DENW094 is already processed, skipping.
		Station DEBW073 is already processed, skipping.
		Station DEHE020 is already processed, skipping.
	 Number of time series meta data fetched : 3
	CSV already exists for timeseries ID 76, skipping download.
Dataframe for timeseries ID 76 loaded successfully with shape (246552, 9)
Station ['DENW094'] min time: 1997-01-01 00:00:00+00:00, max time: 2025-06-27 07:00:00+00:00, hours between: 249703.00
	CSV already exists for timeseries ID 22639, skipping download.
Dataframe for timeseries ID 22639 loaded successfully with shape (110890, 9)
Station ['DEBW073'] min time: 1997-01-01 00:00:00+00:00, max time: 2011-12-31 23:00:00+00:00, hours between: 131471.00
	CSV already exists for timeseries ID 18022, skipping download.
Dataframe for timeseries ID 18022 loaded successfully with shape (220677, 9)
Station ['DEHE020'] min time: 1997-01-01 00:00:00+00:00, max time: 2025-06-18 07:00:00+00:00, hours between: 249487.00
	 Total dataFrames processed :

,longitude,latitude,station_code,datetime,temp
249704,7.567796,47.819182,DEBW073,1997-01-01 00:00:00+00:00,-10.0
249705,7.567796,47.819182,DEBW073,1997-01-01 01:00:00+00:00,-11.0
249706,7.567796,47.819182,DEBW073,1997-01-01 02:00:00+00:00,-11.0
249707,7.567796,47.819182,DEBW073,1997-01-01 03:00:00+00:00,-12.0
249708,7.567796,47.819182,DEBW073,1997-01-01 04:00:00+00:00,-12.0


Most of the data in TOAR is observational data which has few or many data gaps. The "temp"(erature) data that you downloaded is actually from a model (ERA5 reanalysis) and should therefore not have any missing values (except if your requested daterange extends beyond the period for which we extracted ERA5 data).

To demonstrate a more typical workflow of handling observational data, we will now inspect, interpolate, and remove missing data values. Note that the pivot routine above merged data from different variables together and introduced NaNs.

## Data handling (observational gaps)

😈 **Task 4:** Insert some code to find out how many NaN values each variable in the dataset contains. Even better: try to generate a bargraph showing the number of instances with consecutive NaN values (1, 2, 3, ..., more than 10).

To provide as much training data to ML models as possible, you typically want to fill gaps through interpolation. For environmental data, it is common practice to interpolate over up to 6 hours but not longer. In many cases, linear interpolation will work just fine.


😈 **Question 3:** Why is it acceptable to fill up to 6 missing hourly values in this dataset?

😈 **Question 4:** Why do you need to interpolate at all? Wouldn't it be sufficient to randomly sample from the existing data?


In [161]:
import numpy as np
def fill_six_nans(group):
    """
    Fills up to six consecutive NaN values in a given pandas Series using linear interpolation
    if the NaNs are surrounded by valid values. If the NaNs are at the start, they are replaced
    with zeros, and if they are at the end, they are filled with the last known value.

    Args:
        group (pd.Series): The input Series with potential NaN values.

    Returns:
        pd.Series: A Series where up to six consecutive NaNs are interpolated, and longer NaN
        sequences are partially filled while preserving the original index.
    """
    values = group.to_numpy()
    i = 0
    while i < len(values):
        if np.isnan(values[i]):
            start = i
            while i < len(values) and np.isnan(values[i]):
                i += 1
            end = min(i, start + 6)  # Limit to filling only 6 NaNs

            if start > 0 and i < len(values):  # NaNs in the middle
                fill_values = np.linspace(values[start - 1], values[i], end - start + 2)[1:-1]
            elif start == 0:  # NaNs at the start
                fill_values = [0] * (end - start)
            elif i >= len(values):  # NaNs at the end
                fill_values = [values[start - 1]] * (end - start)
            values[start:end] = fill_values
        else:
            i += 1
    return pd.Series(values, index=group.index)

In [48]:
dataframes[variable_columns] = dataframes.groupby(group_columns)[variable_columns].transform(fill_six_nans)
dataframes.isna().sum()

longitude             0
latitude              0
station_code          0
datetime              0
o3              1316760
no2             1317706
no              1319226
temp              62161
press           1367210
dtype: int64

Now rest of the Nas can be dropped as that station might not have data collected in the time period and the data needs to be normalized.

😈 **Question 5:** What risks might arise if normalization is applied *before* handling missing values?


In [50]:
dataframes = dataframes.dropna()
dataframes.isna().sum()

longitude       0
latitude        0
station_code    0
datetime        0
o3              0
no2             0
no              0
temp            0
press           0
dtype: int64

In [51]:
dataframes.shape

(0, 9)

## Staged data loading (Housekeeping)

Finally, and before we want to perform any analysis of the data, we will store the "cleaned" dataset for later re-use. Subsequent notebooks can then easily reload the "raw_data" from the local storage or your mounted google drive folder when needed.

In [53]:
dataframes.to_csv(os.path.join(TIMESERIES_DATA_DIR, "raw_data.csv"), index=False)

😈 **Task 6:** Re-inspect the code above and reflect on the data management strategy employed here. What happens if you re-run the script after changing station codes or variables? Would the notebook repeat all download operations if you add a single station or variable? What could you do to improve this workflow?

Don't waste time by trying to build the ultimate do-it-all dataloader - you can attend my lecture on Earth System Data Processing if you are keen to go deeper down this route.

### Leftovers from first cleanup


Below cell can be used to reload data if using an open source notebook servers like google colab and the if the usage limit is reached or for other issues

In [56]:
import pandas as pd
import os

## Raw data csv is also made available for the select stations in URL:
url = "https://drive.google.com/uc?export=download&id=1cmTTWY3f18SikgRBcZzhtFswIf7XwPJq"
dataframes = pd.read_csv(url,parse_dates=["datetime"])
## Else if using local files:
# dataframes = pd.read_csv(os.path.join(TIMESERIES_DATA_DIR, "raw_data.csv"))
dataframes.head()

,longitude,latitude,station_code,datetime,temp
0,7.567796,47.819182,DEBW073,1997-01-01 00:00:00+00:00,-10.0
1,7.567796,47.819182,DEBW073,1997-01-01 01:00:00+00:00,-11.0
2,7.567796,47.819182,DEBW073,1997-01-01 02:00:00+00:00,-11.0
3,7.567796,47.819182,DEBW073,1997-01-01 03:00:00+00:00,-12.0
4,7.567796,47.819182,DEBW073,1997-01-01 04:00:00+00:00,-12.0


In [57]:
dataframes.shape

(579480, 5)

In [58]:
dataframes.isna().sum()

longitude       0
latitude        0
station_code    0
datetime        0
temp            0
dtype: int64

### (Optional/Advanced) Multi-Variable Case:

We've seen how to download one variable for multiple stations. Now let's try and download multi-variables for one station.  

- You know the drill at this point:
  - Set the paths for loading the timeseries_ids(unique to each variable) corresponding to the station code and downloading timeseries data for all the variables.
  - Fetch the variables for the station of interest
  - Fill the gaps upto 6hrs
  - Drop NAs


#### Fetch the variables

In [80]:
# Let us focus on one single station
station_codes = ["DENW094"]
#variable_columns = ["no2", "temp", "o3", "no", "press"]
variable_columns = ["temp", "o3"]
# Note: the TOAR API sets limits to anonymous users; you can download max 3 timeseries with one request.
# Hence, you will need to adapt this code to load data of all 5 variables.


</details>

#### Download the variables data for that 1 station via REST API

In [144]:
# Load existing timeseries IDs from json to skip calls to TOAR
existing_timeseries = load_existing_timeseries_ids()

timeseries_data = fetch_timeseries_data(station_codes, existing_timeseries,variable_columns)
print(f"\t Number of time series meta data fetched : {len(timeseries_data)}")

# save existing timeseries IDs as json to reduce calls to TOAR in future
save_timeseries_ids(timeseries_data)

dataframes_original = download_csv_data(timeseries_data,variable_columns)
print(f"\t Total dataFrames processed : {len(dataframes_original)} and shape of first dataframe {dataframes_original.shape}.")

dataframes_original.head()


		Station DENW094 is already processed, skipping.
	 Number of time series meta data fetched : 2
	CSV already exists for timeseries ID 73, skipping download.
Dataframe for timeseries ID 73 loaded successfully with shape (211247, 9)
Station ['DENW094'] min time: 1999-07-02 18:00:00+00:00, max time: 2025-06-27 07:00:00+00:00, hours between: 227797.00
	CSV already exists for timeseries ID 76, skipping download.
Dataframe for timeseries ID 76 loaded successfully with shape (246552, 9)
Station ['DENW094'] min time: 1997-01-01 00:00:00+00:00, max time: 2025-06-27 07:00:00+00:00, hours between: 249703.00
	 Total dataFrames processed : 477502 and shape of first dataframe (477502, 6).


,longitude,latitude,station_code,datetime,o3,temp
227798,6.093923,50.754704,DENW094,1997-01-01 00:00:00+00:00,NaN,-15.85
227799,6.093923,50.754704,DENW094,1997-01-01 01:00:00+00:00,NaN,-16.35
227800,6.093923,50.754704,DENW094,1997-01-01 02:00:00+00:00,NaN,-16.45
227801,6.093923,50.754704,DENW094,1997-01-01 03:00:00+00:00,NaN,-17.15
227802,6.093923,50.754704,DENW094,1997-01-01 04:00:00+00:00,NaN,-17.35


#### Data handling (observational gaps)

##### Filling up upto 6 NAs

In [147]:
#dataframes = dataframes_original.drop(columns=["no2", "press", "no"])
dataframes = dataframes_original
dataframes.head()

,longitude,latitude,station_code,datetime,o3,temp
227798,6.093923,50.754704,DENW094,1997-01-01 00:00:00+00:00,NaN,-15.85
227799,6.093923,50.754704,DENW094,1997-01-01 01:00:00+00:00,NaN,-16.35
227800,6.093923,50.754704,DENW094,1997-01-01 02:00:00+00:00,NaN,-16.45
227801,6.093923,50.754704,DENW094,1997-01-01 03:00:00+00:00,NaN,-17.15
227802,6.093923,50.754704,DENW094,1997-01-01 04:00:00+00:00,NaN,-17.35


In [148]:
import numpy as np
def fill_six_nans2(group):
    """
    Fills up to six consecutive NaN values in a given pandas Series using linear interpolation
    if the NaNs are surrounded by valid values. If the NaNs are at the start, they are replaced
    with zeros, and if they are at the end, they are filled with the last known value.

    Args:
        group (pd.Series): The input Series with potential NaN values.

    Returns:
        pd.Series: A Series where up to six consecutive NaNs are interpolated, and longer NaN
        sequences are partially filled while preserving the original index.
    """
    values = group.to_numpy()
    i = 0
    while i < len(values):
        if np.isnan(values[i]):
            start = i
            while i < len(values) and np.isnan(values[i]):
                i += 1
            end = min(i, start + 6)  # Limit to filling only 6 NaNs

            if start > 0 and i < len(values):  # NaNs in the middle
                fill_values = np.linspace(values[start - 1], values[i], end - start + 2)[1:-1]
            elif start == 0:  # NaNs at the start
                fill_values = [0] * (end - start)
            elif i >= len(values):  # NaNs at the end
                fill_values = [values[start - 1]] * (end - start)
            values[start:end] = fill_values
        else:
            i += 1
    return pd.Series(values, index=group.index)

In [149]:
dataframes.isna().sum()

longitude            0
latitude             0
station_code         0
datetime             0
o3              266255
temp            230950
dtype: int64

In [150]:
dataframes.shape

(477502, 6)

In [151]:
# Grouping columns
group_cols = ['longitude', 'latitude', 'station_code', 'datetime']

# Aggregate with mean (NaNs ignored automatically)
agg_dict = {
    'temp': 'mean',
    'o3': 'mean'
}
# we have lots of duplicate data regarding (datetime ^ station ^ latitude ^ longitude), here we aggregate them and remove the duplicates, 
# by taking the mean value of duplicates moving forward
dataframes = dataframes.groupby(group_cols, as_index=False).agg(agg_dict)

print(dataframes)

        longitude   latitude station_code                  datetime    temp  \
0        6.093923  50.754704      DENW094 1997-01-01 00:00:00+00:00 -15.850   
1        6.093923  50.754704      DENW094 1997-01-01 01:00:00+00:00 -16.350   
2        6.093923  50.754704      DENW094 1997-01-01 02:00:00+00:00 -16.450   
3        6.093923  50.754704      DENW094 1997-01-01 03:00:00+00:00 -17.150   
4        6.093923  50.754704      DENW094 1997-01-01 04:00:00+00:00 -17.350   
...           ...        ...          ...                       ...     ...   
249699   6.093923  50.754704      DENW094 2025-06-27 03:00:00+00:00  18.200   
249700   6.093923  50.754704      DENW094 2025-06-27 04:00:00+00:00  17.987   
249701   6.093923  50.754704      DENW094 2025-06-27 05:00:00+00:00  17.798   
249702   6.093923  50.754704      DENW094 2025-06-27 06:00:00+00:00  16.891   
249703   6.093923  50.754704      DENW094 2025-06-27 07:00:00+00:00  15.351   

               o3  
0             NaN  
1          

In [152]:
#dropping duplicates just in case, we do not have duplicates at this point though
dataframes = dataframes.drop_duplicates()

In [153]:
dataframes.isna().sum()

longitude           0
latitude            0
station_code        0
datetime            0
temp             3152
o3              38457
dtype: int64

In [154]:
dataframes.shape

(249704, 6)

In [155]:
#simple check how often 1, 2, 3, .. hours time difference appears in the data
df = dataframes
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.sort_values(['station_code', 'datetime']).reset_index(drop=True)

# Compute time differences in hours
time_diffs = df.groupby('station_code')['datetime'].diff()
time_diffs_hours = time_diffs.dt.total_seconds() / 3600

# Count how often each skip length occurs
histogram = time_diffs_hours.value_counts().sort_index()
print("Histogram of time skips (hours):")
print(histogram)

Histogram of time skips (hours):
datetime
1.0    249703
Name: count, dtype: int64


In [156]:
# fill missing values by interpolation
dataframes[variable_columns] = dataframes.groupby(group_columns)[variable_columns].transform(fill_six_nans2)
dataframes.isna().sum()

longitude           0
latitude            0
station_code        0
datetime            0
temp             2090
o3              27565
dtype: int64

##### After dropping NAs

In [158]:
dataframes.shape

(249704, 6)

In [159]:
#drop missing values
dataframes = dataframes.dropna()
dataframes.isna().sum()

longitude       0
latitude        0
station_code    0
datetime        0
temp            0
o3              0
dtype: int64

#### Statistical Sanity Check

In [161]:
#simple check how often 1, 2, 3, .. hours time difference appears in the data
df = dataframes
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.sort_values(['station_code', 'datetime']).reset_index(drop=True)

# Compute time differences in hours
time_diffs = df.groupby('station_code')['datetime'].diff()
time_diffs_hours = time_diffs.dt.total_seconds() / 3600

# Count how often each skip length occurs
histogram = time_diffs_hours.value_counts().sort_index()
print("Histogram of time skips (hours):")
print(histogram)

Histogram of time skips (hours):
datetime
1.0        221354
2.0            20
3.0            20
4.0            15
5.0             6
6.0             8
7.0             7
8.0             5
9.0             4
10.0            4
11.0            7
12.0            1
13.0           10
14.0            2
15.0            3
16.0            2
17.0            3
18.0            1
19.0            4
20.0            2
21.0           30
22.0            4
23.0            2
24.0            4
25.0            1
26.0            1
27.0            4
29.0            1
30.0            1
32.0            1
33.0            1
39.0            1
41.0            2
42.0            1
43.0            1
46.0            2
49.0            1
51.0            1
52.0            1
53.0            2
55.0            1
56.0            2
59.0            2
61.0            1
63.0            1
64.0            1
70.0            1
74.0            1
78.0            1
83.0            2
84.0            1
85.0            1
121.0           1
153.

In [163]:
dataframes.head()

,longitude,latitude,station_code,datetime,temp,o3
0,6.093923,50.754704,DENW094,1997-01-01 00:00:00+00:00,-15.85,0.0
1,6.093923,50.754704,DENW094,1997-01-01 01:00:00+00:00,-16.35,0.0
2,6.093923,50.754704,DENW094,1997-01-01 02:00:00+00:00,-16.45,0.0
3,6.093923,50.754704,DENW094,1997-01-01 03:00:00+00:00,-17.15,0.0
4,6.093923,50.754704,DENW094,1997-01-01 04:00:00+00:00,-17.35,0.0


In [164]:
stats = ['min', 'max', 'mean', 'sum', 'std', 'var', 'median','prod','nunique',
    ('5th_percentile', lambda x: x.quantile(0.05)),
    ('10th_percentile', lambda x: x.quantile(0.10)),
    ('25th_percentile', lambda x: x.quantile(0.25)),
    ('50th_percentile', lambda x: x.quantile(0.50)), #(median)
    ('75th_percentile', lambda x: x.quantile(0.75))]
agg_dict = {col: stats for col in variable_columns}
grouped = dataframes.groupby('station_code').agg(agg_dict)
display(grouped)

for agg_func in ['min', 'max', 'mean', 'std']:
    display(agg_func)
    agg_view = grouped.xs(agg_func, axis=1, level=1)
    display(agg_view)

temp                                                        \
                min     max       mean           sum       std        var   
station_code                                                                
DENW094      -17.35  39.126  10.645179  2.358588e+06  7.319994  53.582319   

                                                 ...         o3              \
             median prod nunique 5th_percentile  ...        std         var   
station_code                                     ...                          
DENW094       10.45  NaN   51018          -0.75  ...  14.646173  214.510397   

                                                                    \
                median prod nunique 5th_percentile 10th_percentile   
station_code                                                         
DENW094       24.56076 -0.0  103270        1.00248        4.448505   

                                                              
             25th_percentile 50th_percentile 75th_percentile  
station_code                                                  
DENW094             14.42916        24.56076       32.921443  

[1 rows x 28 columns]

'min'

,temp,o3
station_code,,
DENW094,-17.35,-1.417507


'max'

,temp,o3
station_code,,
DENW094,39.126,134.83356


'mean'

,temp,o3
station_code,,
DENW094,10.645179,24.565799


'std'

,temp,o3
station_code,,
DENW094,7.319994,14.646173


In [165]:
dataframes.to_csv(os.path.join(TIMESERIES_DATA_DIR, "raw_data.csv"), index=False)

#### Log Scaling after checking for skewdness

*ToDo:* This cell should be transfered to the data preparation part of the ML models.




In [167]:
import numpy as np
from scipy.stats import skew

def log_transform_if_skewed(df, columns, threshold=1.0):
    """
    Log-transform the specified columns of a DataFrame based on their skewdness.

    Args:
        df (pd.DataFrame): The input DataFrame.
        columns (list): List of column names that need to be checked for skewdness.

    Returns:
        pd.DataFrame: DataFrame with normalized columns.
    """
    df_transformed = df.copy()

    for col in columns:
        # s = df[col].dropna()
        s = df[col]
        current_skewness = skew(s)

        print(f"[{col}] Skewness: {current_skewness:.2f}")

        if abs(current_skewness) > threshold:
            # To avoid log(0) or log(negative values).
            if (s <= 0).any():
                shift = abs(s.min()) + 1e-6
                print(f"Applying log(x + {shift:.6f}) to {col}")
                df_transformed[col] = np.log(df[col] + shift)
            else:
                print(f"Applying log(x) to {col}")
                df_transformed[col] = np.log(df[col])
        else:
            print(f"No transformation applied to {col}.")

    return df_transformed

In [168]:
dataframe_= log_transform_if_skewed(dataframes, variable_columns, threshold=1.0)
dataframe_.head()

[temp] Skewness: 0.17
No transformation applied to temp.
[o3] Skewness: 0.62
No transformation applied to o3.


,longitude,latitude,station_code,datetime,temp,o3
0,6.093923,50.754704,DENW094,1997-01-01 00:00:00+00:00,-15.85,0.0
1,6.093923,50.754704,DENW094,1997-01-01 01:00:00+00:00,-16.35,0.0
2,6.093923,50.754704,DENW094,1997-01-01 02:00:00+00:00,-16.45,0.0
3,6.093923,50.754704,DENW094,1997-01-01 03:00:00+00:00,-17.15,0.0
4,6.093923,50.754704,DENW094,1997-01-01 04:00:00+00:00,-17.35,0.0


#### Normalize the data

😈 Task 8: Why do we need both log transformation and Z-score normalization?

In [171]:
dataframes.to_csv("not_normalized_data_multi.csv", index=False)

In [172]:
def standard_scaler(df, columns):
    """
    Standardize the specified columns of a DataFrame by subtracting the mean
    and dividing by the standard deviation (Z-score normalization).

    Args:
        df (pd.DataFrame): The input DataFrame.
        columns (list): List of column names to be normalized.

    Returns:
        pd.DataFrame: DataFrame with normalized columns.
    """
    df_scaled = df.copy()
    for col in columns:
        mean = df_scaled[col].mean()
        print(mean)
        std = df_scaled[col].std()
        df_scaled[col] = (df_scaled[col] - mean) / std
    return df_scaled

In [173]:
dataframes = standard_scaler(dataframe_, variable_columns)
dataframes.head()

10.645179247129887
24.565799430301844


,longitude,latitude,station_code,datetime,temp,o3
0,6.093923,50.754704,DENW094,1997-01-01 00:00:00+00:00,-3.619563,-1.677284
1,6.093923,50.754704,DENW094,1997-01-01 01:00:00+00:00,-3.687869,-1.677284
2,6.093923,50.754704,DENW094,1997-01-01 02:00:00+00:00,-3.701530,-1.677284
3,6.093923,50.754704,DENW094,1997-01-01 03:00:00+00:00,-3.797159,-1.677284
4,6.093923,50.754704,DENW094,1997-01-01 04:00:00+00:00,-3.824481,-1.677284


##### Save the normalized dataframe for later use

In [175]:
dataframes.to_csv("normalized_data_multi.csv", index=False)

In [176]:
dataframes.shape

(221564, 6)